### Cell 1

In [39]:
import pandas as pd

df = pd.read_csv('Dataset_B.csv')
df['item_count'] = 1                #put 1 for each item, making it countable for item counts
df.dtypes                           #check datatypes for all columns

Member_number       int64
Date               object
itemDescription    object
item_count          int64
dtype: object

### Cell 2

In [40]:
df['Member_number'] = df['Member_number'].astype(str)   #convert member number to string as we do not wish to perform any mathematical computation. Only identification
df.dtypes                                               #check if data type has been changed

Member_number      object
Date               object
itemDescription    object
item_count          int64
dtype: object

### Cell 3

In [41]:
df.describe(include='all')  #statistical summary of the whole dataframe

,Member_number,Date,itemDescription,item_count
count,19415,19415,19415,19415.0
unique,3814,728,164,NaN
top,3308,28-08-2014,whole milk,NaN
freq,19,54,957,NaN
mean,NaN,NaN,NaN,1.0
std,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,1.0
25%,NaN,NaN,NaN,1.0
50%,NaN,NaN,NaN,1.0
75%,NaN,NaN,NaN,1.0


### Cell 4

In [42]:
df_table = pd.DataFrame(df.groupby('itemDescription')['item_count'].sum().sort_values(ascending=False).reset_index())   #create new dataframe with item counts sorted in descending order
df_table.head().style.background_gradient(cmap='Greens')                                                                #view first 5 rows of dataframe

,itemDescription,item_count
0,whole milk,957
1,rolls/buns,901
2,soda,863
3,yogurt,774
4,other vegetables,752


### Cell 5

In [43]:
import plotly.express as px         #import library for visualization

df_table['all'] = 'Top 50 items'    #ensure whole dataframe has the same origin in the treemap

#creating a treemap for the top 50 items bought
fig = px.treemap(df_table.head(50), path=['all', 'itemDescription'], values='item_count', color=df_table['item_count'].head(50), hover_data=['itemDescription'], color_continuous_scale='Greens')
fig.show()

### Cell 6

In [44]:
item_dict = dict()                                  #dictionary to store the grouped data
                                                    #loop through each row of the dataframe
for i in df.values:                                 #store each row in a list format
    i = list(i)                                     #check if (i[0]=Member_number) is already present in the dictionary  
    if i[0] in item_dict.keys():                    #check if (i[1]=Date) is already present under the Member_number
        if i[1] in item_dict[i[0]]:                 #if i[0] and i[1] are both present, append the items bought on that day
            item_dict[i[0]][i[1]].append(i[2])      
        else:
            item_dict[i[0]].update({i[1]:[i[2]]})   #if i[1] is not present in in i[0], update the dictionary with the date and item bought
    else:
        item_dict[i[0]] = {i[1]:[i[2]]}             #if Member_number is not present in the dictionary, create a new entry with Member_number, Date and item_bought

item_dict

{'3562': {'18-03-2015': ['salty snack'],
  '13-08-2015': ['whole milk', 'soda', 'canned beer', 'hygiene articles'],
  '30-12-2015': ['shopping bags', 'rolls/buns'],
  '11-09-2014': ['white wine'],
  '17-12-2014': ['male cosmetics', 'newspapers']},
 '3145': {'16-11-2015': ['cake bar'],
  '15-08-2014': ['whole milk'],
  '16-08-2014': ['candy']},
 '3595': {'17-12-2015': ['whole milk'],
  '06-08-2015': ['other vegetables'],
  '15-10-2015': ['zwieback'],
  '21-03-2015': ['cream cheese ', 'waffles'],
  '06-01-2014': ['canned beer', 'bottled water', 'waffles'],
  '17-01-2014': ['curd']},
 '4934': {'17-03-2015': ['other vegetables'], '13-10-2014': ['bottled beer']},
 '3386': {'03-02-2015': ['yogurt'],
  '07-08-2014': ['baking powder'],
  '18-03-2014': ['pasta'],
  '10-05-2014': ['soda']},
 '4125': {'21-03-2015': ['yogurt'],
  '20-03-2014': ['brown bread', 'candy'],
  '03-06-2014': ['rolls/buns'],
  '21-09-2014': ['pastry', 'pet care']},
 '2570': {'19-03-2015': ['roll products '],
  '24-08-2015

### Cell 7

In [45]:
item_length = 0

for i in item_dict:
    for j in item_dict[i]:
        item_length = item_length + len(item_dict[i][j])

print('Total items in the dictionary are:', item_length)

Total items in the dictionary are: 19415


### Cell 8

In [46]:
transactions = list()
transactions_length = 0

for i in item_dict:
    for j in item_dict[i]:
        transactions.append(tuple(set(item_dict[i][j])))

for i in transactions:
    transactions_length = transactions_length + len(i)

print('Total items in the tuple accounting for duplicates are:', transactions_length)

Total items in the tuple accounting for duplicates are: 19250


### Cell 9

In [47]:
maximum = (df_table['item_count'].loc[50])/transactions_length
minimum = 0
interval = (maximum - minimum)/10
minimum_support = list()

for i in range(9):
    minimum = minimum + interval
    minimum_support.append(round(minimum, 4))

minimum_support

[0.0005, 0.0011, 0.0016, 0.0022, 0.0027, 0.0032, 0.0038, 0.0043, 0.0049]

### Cell 10

In [48]:
from efficient_apriori import apriori

support_dict = dict()
 
min_confidence = 0      # For now set min_confidence = 0 to obtain all the rules

for support in minimum_support:
    itemsets, rules = apriori(transactions, min_support=support, min_confidence=min_confidence)
    support_dict[support] = len(rules)

support_dict

{0.0005: 478,
 0.0011: 94,
 0.0016: 30,
 0.0022: 14,
 0.0027: 0,
 0.0032: 0,
 0.0038: 0,
 0.0043: 0,
 0.0049: 0}

### Cell 11

In [49]:
from efficient_apriori import apriori

min_support = 0.0005
min_confidence = 0      # For now set min_confidence = 0 to obtain all the rules

itemsets, rules = apriori(transactions, min_support=min_support, min_confidence=min_confidence)

### Cell 12

In [50]:
Rules = list()
Confidence = list()
Support = list()
Lift = list()
Conviction = list()

for rule in rules:
    rule = str(rule)

    Rules.append(rule[0:(rule.rfind('}')) + 1])
    Confidence.append(rule[(rule.index('conf: ')) + 5:(rule.index('conf: ')) + 11])
    Support.append(rule[(rule.index('supp: ')) + 5:(rule.index('supp: ')) + 11])
    Lift.append(rule[(rule.index('lift: ')) + 5:(rule.index('lift: ')) + 11])
    Conviction.append(rule[(rule.index('conv: ')) + 5:(rule.index('conv: ')) + 11])

df_rules = pd.DataFrame({'Rules':Rules, 'Confidence':Confidence, 'Support':Support, 'Lift':Lift, 'Conviction':Conviction})

df_rules

,Rules,Confidence,Support,Lift,Conviction
0,{soda} -> {beverages},0.009,0.001,1.155,1.001
1,{beverages} -> {soda},0.077,0.001,1.155,1.011
2,{bottled water} -> {bottled beer},0.015,0.001,0.536,0.987
3,{bottled beer} -> {bottled water},0.023,0.001,0.536,0.980
4,{brown bread} -> {bottled beer},0.027,0.001,0.980,0.999
...,...,...,...,...,...
473,{white bread} -> {whole milk},0.039,0.001,0.534,0.964
474,{yogurt} -> {white bread},0.012,0.001,0.591,0.992
475,{white bread} -> {yogurt},0.035,0.001,0.591,0.975
476,{yogurt} -> {whole milk},0.030,0.002,0.408,0.955


### Cell 13

In [51]:
df_rules['Confidence'] = df_rules['Confidence'].astype(float) 
df_rules['Support'] = df_rules['Support'].astype(float) 
df_rules['Lift'] = df_rules['Lift'].astype(float) 
df_rules['Conviction'] = df_rules['Conviction'].astype(float) 
df_rules.dtypes

Rules          object
Confidence    float64
Support       float64
Lift          float64
Conviction    float64
dtype: object

### Cell 14

In [52]:
df_rules.drop(df_rules[df_rules['Lift'] <= 1].index, inplace=True) 
df_rules.drop(df_rules[df_rules['Conviction'] == 1].index, inplace=True) 
df_rules.describe()

,Confidence,Support,Lift,Conviction
count,54.000000,54.000,54.00000,54.000000
mean,0.036481,0.001,1.27013,1.007444
std,0.022948,0.000,0.24342,0.007820
min,0.007000,0.001,1.01500,1.001000
25%,0.018250,0.001,1.08350,1.002000
50%,0.030500,0.001,1.15500,1.004000
75%,0.052500,0.001,1.42850,1.010750
max,0.090000,0.001,1.88300,1.033000


### Cell 15

In [53]:
df_rules.sort_values(by=['Conviction', 'Lift', 'Confidence'], ascending=[False, False, False], inplace=True)
df_rules = df_rules.reset_index(drop=True)
df_rules = df_rules[0:15]
df_rules

,Rules,Confidence,Support,Lift,Conviction
0,{detergent} -> {yogurt},0.090,0.001,1.501,1.033
1,{soft cheese} -> {yogurt},0.087,0.001,1.457,1.030
2,{cat food} -> {fruit/vegetable juice},0.054,0.001,1.813,1.026
3,{chewing gum} -> {newspapers},0.056,0.001,1.660,1.023
4,{napkins} -> {pastry},0.059,0.001,1.566,1.023
5,{long life bakery product} -> {napkins},0.040,0.001,1.883,1.020
6,{sugar} -> {bottled water},0.058,0.001,1.343,1.016
7,{napkins} -> {long life bakery product},0.030,0.001,1.883,1.014
8,{sugar} -> {coffee},0.037,0.001,1.564,1.014
9,{pastry} -> {napkins},0.033,0.001,1.566,1.012


### Cell 16

In [54]:
#Export association rules to .xlsx
df_rules.to_excel(f'Association_Rules.xlsx', index=False)

### Cell 17

In [55]:
min_support = 0.0022
min_confidence = 0      # For now set min_confidence = 0 to obtain all the rules

itemsets, rules = apriori(transactions, min_support=min_support, min_confidence=min_confidence)

Rules = list()
Confidence = list()
Support = list()
Lift = list()
Conviction = list()

for rule in rules:
    rule = str(rule)

    Rules.append(rule[0:(rule.rfind('}')) + 1])
    Confidence.append(rule[(rule.index('conf: ')) + 5:(rule.index('conf: ')) + 11])
    Support.append(rule[(rule.index('supp: ')) + 5:(rule.index('supp: ')) + 11])
    Lift.append(rule[(rule.index('lift: ')) + 5:(rule.index('lift: ')) + 11])
    Conviction.append(rule[(rule.index('conv: ')) + 5:(rule.index('conv: ')) + 11])

df_rules = pd.DataFrame({'Rules':Rules, 'Confidence':Confidence, 'Support':Support, 'Lift':Lift, 'Conviction':Conviction})

df_rules['Confidence'] = df_rules['Confidence'].astype(float) 
df_rules['Support'] = df_rules['Support'].astype(float) 
df_rules['Lift'] = df_rules['Lift'].astype(float) 
df_rules['Conviction'] = df_rules['Conviction'].astype(float)

df_rules.sort_values(by=['Conviction', 'Lift', 'Confidence'], ascending=[False, False, False], inplace=True)
df_rules = df_rules.reset_index(drop=True)
df_rules = df_rules[0:15]
df_rules

,Rules,Confidence,Support,Lift,Conviction
0,{soda} -> {shopping bags},0.035,0.002,0.761,0.988
1,{rolls/buns} -> {shopping bags},0.033,0.002,0.710,0.986
2,{shopping bags} -> {soda},0.051,0.002,0.761,0.983
3,{shopping bags} -> {rolls/buns},0.049,0.002,0.710,0.979
4,{soda} -> {yogurt},0.039,0.003,0.647,0.978
5,{yogurt} -> {soda},0.043,0.003,0.647,0.975
6,{whole milk} -> {other vegetables},0.033,0.002,0.562,0.973
7,{other vegetables} -> {whole milk},0.041,0.002,0.562,0.966
8,{rolls/buns} -> {soda},0.034,0.002,0.512,0.966
9,{whole milk} -> {soda},0.034,0.003,0.511,0.966
